# GitFlow - Release

This notebook assumes you've gone through the previous notebooks, especially

- [Create Projects](11_create_projects.ipynb)
- [GitFlow First Steps](21_gitflow_first_steps.ipynb)

This notebooks prepares `lib` and `app` so that `app`'s `develop` branch consumes the "latest and greatest" API compatible version of `lib` (e.g. `1.x`).

It then creates a `release` of `lib` and `app` and ensures that the release of `app` consumes the release version of `lib` rather then the `develop` version.

## Preparation
As shown in preceeding notebooks, we first have to build the infrastructure. This notebook assumes the reader has worked through the other notebook and therefore it's not being explained here again.

In [ ]:
from cicd_sim import *
bitbucket = Repos()
artifactory = Artifactory()
jenkins = Jenkins(artifactory, bitbucket)
# ensure 'random' generates the same number each time this notebook is run
import random
random.seed(123)

## Setup Repos

As done in the preceeding notebooks, prepare the repos.

In [ ]:
lib = bitbucket.create_repo('lib')
lib_dev = lib.checkout('develop', 'magenta')
lib_dev.set_version('1.2.0')

app = bitbucket.create_repo('app')
app_dev = app.checkout('develop', 'cyan')
app_dev.set_version('4.0.0')
app_dev.set_requires('lib/1.x')

## Versioning with pre-releases

The benefit of using pre-releases is that no merge conflicts will happen.

In [ ]:
lib_dev.merge(lib_dev.checkout('feature/a').commit_file('feature a', 'aaaaa')).push()

In [ ]:
lib_dev.merge(lib_dev.checkout('feature/b').commit_file('feature b', 'bbbbb')).push()

Similar stuff's happening on `app`...

In [ ]:
# multiple developers start working on feature branches...
app_dev.merge(app_dev.checkout('feature/a').commit_file('feature a', 'aaaaa').commit_file('feature a2', '2222222')).push()
app_dev.merge(app_dev.checkout('feature/b').commit_file('feature b', 'bbbbb')).push()
app_dev.merge(app_dev.checkout('feature/c').commit_file('feature c', 'ccccc')).push()

## Application Version

Now, we want to release `app`. Therefore, we first release `lib`...

See how the published `lib` release candidate (`rc`) will immediately be picked up the `app`'s `develop` branch which was setup to consume 'latest and greatest':

In [ ]:
lib_release = lib_dev.checkout('release/1.2.0', 'yellow')
lib_release.push()

## Library Version

While `lib` is going to be stabilized for being released, `lib` developement will continue on `develop`.

In [ ]:
lib_dev.merge(lib_dev.checkout('feature/showMustGoOn').commit_file('ShowMustGo', 'ON!')).push()

## Oooops

The developers at `lib` forgot to increase `lib` version to work "towards the next release". The "next release" may contain "additional features" or "API changes"... depending on that decision, the version must be adjusted to either `1.3.0` or `2.0.0`.

Note that although the version has not been adjusted by mistake, `app` still uses the "better" version, which is the "realease candidate" (`rc`) in this case. Therefore: It didn't do any harm...

In [ ]:
lib_next_version = lib_dev.checkout('feature/IncreaseVersion')
lib_next_version.set_version('1.3.0')
lib_dev.merge(lib_next_version).push()

## Create `app` release

Now it's time to creat a release of `app`...

In [ ]:
app_release = app_dev.checkout('release/4.0.0', 'yellow')
app_release.set_requires('lib/>1.2.0-0 <1.3.0-0')
app_release.push()

## Bugfix on `lib`

`lib` needs bug fixing...

In [ ]:
lib_release.commit_file('bugfix', 'fixed!').push()

## `lib` released

In [ ]:
lib_master = lib.checkout('master', 'green')
lib_master.merge(lib_release).push()

## `app` released

In [ ]:
app_master = app.checkout('master', 'green')
app_master.merge(app_release).push()

## Working on `app`'s `develop` continues

In [ ]:
app_dev.set_version('4.1.0')
app_dev.push()

## Hotfix on `app`

In [ ]:
app_hotfix = app_master.checkout('hotfix/4.0.1', 'yellow')
app_hotfix.set_version('4.0.1')
app_hotfix.commit_file('fixit', 'done.')
app_hotfix.push()

In [ ]:
app_master.merge(app_hotfix).push()